<a href="https://colab.research.google.com/github/Fujisupakon/NIH-CHEST-XRAY/blob/main/Aixray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
import torch
from torchvision import models, transforms
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CSV_PATH = "Data_Entry_2017.csv"
IMAGE_PATH = "/content/drive/MyDrive/AI xray/unseenimage"

In [ ]:
MODEL_PATH = "nih_model.pth"
THRESHOLD = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#load deseases and keep it in list
df = pd.read_csv(CSV_PATH)
all_labels = df["Finding Labels"].str.split("|").explode().unique()
LABELS = sorted([label for label in all_labels if isinstance(label, str) and label != "No Finding"])
print("Loaded Labels:", LABELS)

Loaded Labels: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [ ]:
#prepare image for model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
#load model and weights
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(LABELS))
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#predict
def predict_batch(image_folder):
    image_paths = glob.glob(os.path.join(image_folder, "*"))
    predictions = []

    for image_path in image_paths:
        image = Image.open(image_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.sigmoid(output).cpu().squeeze()
            preds = (probs >= THRESHOLD).int()

        results = {label: bool(preds[i]) for i, label in enumerate(LABELS)}
        predictions.append((os.path.basename(image_path), results))

    return predictions

In [ ]:
#display prediction
result = predict_batch(IMAGE_PATH)

for filename, prediction in result:
    print(f"\n{filename}")
    for label, present in prediction.items():
        #print(f"  {label}: {'Yes' if present else 'No'}")
        if present:
            print(f"{label} : YES")
        else:
            print(f"{label} : NO")


Atelectasis_wikipedia.jpg
Atelectasis : YES
Cardiomegaly : NO
Consolidation : NO
Edema : NO
Effusion : NO
Emphysema : NO
Fibrosis : NO
Hernia : NO
Infiltration : NO
Mass : NO
Nodule : NO
Pleural_Thickening : NO
Pneumonia : NO
Pneumothorax : YES

pneumothorax x ray wiki.jpg
Atelectasis : NO
Cardiomegaly : NO
Consolidation : NO
Edema : NO
Effusion : NO
Emphysema : NO
Fibrosis : NO
Hernia : NO
Infiltration : NO
Mass : NO
Nodule : NO
Pleural_Thickening : YES
Pneumonia : NO
Pneumothorax : NO
